# Parameter Tuning
* xgboost
* Bayesian 
* GPU ver. (https://eagle705.github.io/articles/2018-06/XGBoost-%EC%A0%95%EB%A6%AC)

In [ ]:
import os
import sys
import joblib
import pickle as pkl

import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore')
warnings.filterwarnings(action='ignore', category=DataConversionWarning)


import pandas as pd 
import numpy as np


from IPython.display import display
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option('display.max_info_columns', 500)

In [ ]:
# !git clone --recursive https://github.com/dmlc/xgboost
# cd xgboost
# !mkdir build
# cd build
# !pip install cmake --upgrade
# !cmake .. -DUSE_CUDA=ON -DUSE_NCCL=ON
# !make -j4
# cd ../python-package
# !sudo python setup.py install
# !sudo apt-get install python-setuptools
# !pip3 install scikit-learn==0.21.3 --upgrade

In [ ]:
# !git clone https://github.com/Microsoft/LightGBM
# cd LightGBM
# !mkdir build
# !cmake -DUSE_GPU=1
# !make -j$(nproc)
# !sudo apt-get -y install python-pip
# !sudo -H pip install setuptools pandas numpy scipy scikit-learn -U
# %cd /content/LightGBM/python-package
# !sudo python setup.py install
# !pip3 uninstall scikit-learn
# !pip3 install scikit-learn==0.21.3

In [ ]:
cd /content/drive/My Drive/쇼핑광고등어/3. Modeling/재빈

[Errno 2] No such file or directory: '/content/drive/My Drive/쇼핑광고등어/3. Modeling/재빈'
/content


In [ ]:
import BoostingModel as model
import SHAPvalue as SHAP
import optimization as opt

In [ ]:
import imp
imp.reload(opt)

<module 'optimization' from '/content/drive/.shortcut-targets-by-id/1cWc-Nky29igOCXv9H2nYRUOItUS3kV8X/쇼핑광고등어/3. Modeling/재빈/optimization.py'>

In [ ]:
imp.reload(SHAP)

<module 'SHAPvalue' from '/content/drive/.shortcut-targets-by-id/1cWc-Nky29igOCXv9H2nYRUOItUS3kV8X/쇼핑광고등어/3. Modeling/재빈/SHAPvalue.py'>

In [ ]:
data_v4 = joblib.load(os.path.join('..', '..', '1. Data', '05_분석데이터', '7th_train_FE.pkl'))
locals().update(data_v4)

In [ ]:
X = data_v4['X'] ; y = data_v4['y']
X.shape, y.shape

((35379, 402), (35379,))

In [ ]:
y2 = np.log1p(y)

## Modeling 

In [ ]:
!pip install bayesian-optimization
!pip install shap 

  Created wheel for bayesian-optimization: filename=bayesian_optimization-1.2.0-cp36-none-any.whl size=11685 sha256=e6ae4af2b3870d05a206aa442c5ae844823bf620f6cf66a243f4a397e16e4802
  Stored in directory: /root/.cache/pip/wheels/5a/56/ae/e0e3c1fc1954dc3ec712e2df547235ed072b448094d8f94aec
Successfully built bayesian-optimization


### xgb_params1

In [ ]:
from bayes_opt import BayesianOptimization

In [ ]:
params_range = {
                'reg_lambda': (1e-3, 1.0),
                'reg_alpha': (1e-3, 1.0),
                'max_depth': (4, 31),
                'eta': (0.001, 0.01),
                'gamma': (1e-3, 1.0),
                'subsample': (1e-3, 1.0),  
                'colsample_bytree': (1e-3, 1.0),  
                'colsample_bylevel': (1e-3, 1.0),  
                'min_child_weight': (1e-3, 10.0), 
                'max_bin': (31, 256)
                }

In [ ]:
def Bayes_running(reg_lambda, reg_alpha, max_depth, gamma, subsample, colsample_bytree, colsample_bylevel, min_child_weight, max_bin, eta):
    params = { 
                'nthread': -1,
                'silent' : 1, 
                'objective': 'reg:linear',
                'eval_metric' : 'rmse',
                'n_estimators': 30000,
                'seed': 77, 
                
                "reg_lambda": max(reg_lambda, 0),
                'reg_alpha': max(reg_alpha, 0),
                "gamma": gamma,
                "max_depth" : int(round(max_depth)),
                'colsample_bytree': colsample_bytree, 
                'colsample_bylevel': colsample_bylevel,
                'min_child_weight': min_child_weight,
                'reg_alpha': max(reg_alpha, 0),
                'subsample': max(min(subsample,1),0), 
                'max_bin': int(round(max_bin)),
              
                "eta": eta, 
                'tree_method' : 'gpu_hist' , 
                'predictor' : 'gpu_predictor'
              
    }
    
    print(params)

    mape, pred = opt.xgb_model(X, y2, params, cv_splits=5, scaling='MinMax', epoch=30000)
    return -(mape['final_mape'][0])

In [ ]:
%%time 

Bayesian = BayesianOptimization(Bayes_running, params_range, random_state = 77)
Bayesian.maximize(init_points = 10, n_iter = 20)

|   iter    |  target   | colsam... | colsam... |    eta    |   gamma   |  max_bin  | max_depth | min_ch... | reg_alpha | reg_la... | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
{'nthread': -1, 'silent': 1, 'objective': 'reg:linear', 'eval_metric': 'rmse', 'n_estimators': 30000, 'seed': 77, 'reg_lambda': 0.240994940692138, 'reg_alpha': 0.5415267536544917, 'gamma': 0.14017525353463228, 'max_depth': 25, 'colsample_bytree': 0.6425534043239316, 'colsample_bylevel': 0.9191899227673827, 'min_child_weight': 3.262183219407854, 'subsample': 0.545877502650329, 'max_bin': 51, 'eta': 0.007783410067562591, 'tree_method': 'gpu_hist', 'predictor': 'gpu_predictor'}
[06:55:46] WARNING: /content/xgboost/src/objective/regression_obj.cu:171: reg:linear is now deprecated in favor of reg:squarederror.
[06:55:46] WARNING: /content/xgboost/src/learner.cc:529: 
Parameters: { n_estimators, silent } 

In [ ]:
print('Final result: ', Bayesian.max)

In [ ]:
# prior은 정확한 관측치로부터 얻어진 것이 아니므로, 대략적인 정수를 사용해도 무방하다고 함!! 
xgb_params1 = Bayesian.max["params"].copy()
xgb_params1["max_depth"] = int(round(xgb_params1["max_depth"]))
xgb_params1["max_bin"] = int(round(lgbm_params1["max_bin"]))

xgb_params1['nthread'] = -1
xgb_params1['silent'] = 1
xgb_params1['n_estimators'] = 30000
xgb_params1['objective'] = 'reg:linear'
xgb_params1['eval_metric'] = 'mape'
xgb_params1['tree_method'] = 'gpu_hist'
xgb_params1['predictor'] = 'gpu_predictor'

In [ ]:
imp.reload(SHAP)

In [ ]:
xgb_mape1, xgb_pred1, xgb_shap1 = SHAP.xgb_SHAP(X, y2, xgb_params1, version='0921-xgb-bayes', cv_splits=5, scaling='MinMax', epoch=30000)

In [ ]:
xgb_mape1